In [1]:
import json
import pandas as pd
import openai
import time
from sklearn.metrics import accuracy_score, classification_report

/Applications/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
file_path = "../data/train.jsonl"

In [14]:
data = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [15]:
# 🔹 Filter: "VERIFIABLE" claims & "SUPPORTS"/"REFUTES" labels
df_filtered = df[(df["verifiable"] == "VERIFIABLE") & (df["label"].isin(["SUPPORTS", "REFUTES"]))].copy()
df_filtered = df_filtered[["claim", "label"]].head(200)
print("\n=== Label Distribution in First 200 Rows ===")
print(df_filtered['label'].value_counts())


=== Label Distribution in First 200 Rows ===
label
SUPPORTS    146
REFUTES      54
Name: count, dtype: int64


In [16]:
def classify_claim(claim_text, model="gpt-4-turbo"):
    """Calls OpenAI API to classify a claim as SUPPORTS or REFUTES."""
    if not isinstance(claim_text, str) or claim_text.strip() == "":
        return None  # Avoid empty inputs
    
    prompt = f"""
    Determine whether the following claim is "SUPPORTS" or "REFUTES".
    Provide only "SUPPORTS" or "REFUTES" as the output.
    
    Claim:
    {claim_text}
    """

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "system", "content": "You are a fact-checking AI."},
                      {"role": "user", "content": prompt}],
            temperature=0
        )
        prediction = response.choices[0].message.content.strip().upper()
        return prediction if prediction in ["SUPPORTS", "REFUTES"] else None
    except Exception as e:
        print(f"Error processing claim: {e}")
        return None

In [ ]:
# 🔹 Run Classification on First 200 Rows
df_filtered["LLM_Prediction"] = df_filtered["claim"].apply(classify_claim)

# 🔹 Remove rows where LLM didn't return a valid prediction
df_filtered.dropna(subset=["LLM_Prediction"], inplace=True)

# 🔹 Convert Labels to Match
df_filtered["label"] = df_filtered["label"].str.upper()
df_filtered["LLM_Prediction"] = df_filtered["LLM_Prediction"].str.upper()

# 🔹 Save results to CSV
output_file = "llm_predictions_200rows.csv"
df_filtered.to_csv(output_file, index=False, encoding="utf-8")
print(f"\nResults saved to {output_file}")



Results saved to llm_predictions_200rows.csv

=== LLM Model Performance ===
Accuracy: 0.8550
              precision    recall  f1-score   support

     REFUTES       0.67      0.93      0.78        54
    SUPPORTS       0.97      0.83      0.89       146

    accuracy                           0.85       200
   macro avg       0.82      0.88      0.83       200
weighted avg       0.89      0.85      0.86       200



In [1]:
# ===================================================
# 🔹 Evaluate LLM Performance
# ===================================================
true_labels = df_filtered["label"].values
predicted_labels = df_filtered["LLM_Prediction"].values

# Compute metrics
accuracy = accuracy_score(true_labels, predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=["REFUTES", "SUPPORTS"])

print("\n=== LLM Model Performance ===")
print(f"Accuracy: {accuracy:.4f}")
print(report)

NameError: name 'df_filtered' is not defined